In [2]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, clear_output

from scipy.integrate import odeint

import matplotlib.pyplot as plt
import numpy as np

import math

from matplotlib import figure
from ipywidgets import interact

import ternary

In [3]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [4]:
# Parameters widgets

tF = widgets.FloatSlider(min=0., max=1., step=0.1, value=1.)
tD = widgets.FloatSlider(min=0., max=1., step=0.1, value=0.3)
tS = widgets.FloatSlider(min=0., max=1., step=0.1, value=0.3)
muftd = widgets.FloatSlider(min=0., max=0.01, step=0.001, value=0.001)
mudtf = widgets.FloatSlider(min=0., max=0.1, step=0.01, value=0.01)
mufts = widgets.FloatSlider(min=0., max=0.01, step=0.0001, value=0.0001)
mustf = widgets.FloatSlider(min=0., max=0.01, step=0.0001, value=0.0001)
Kwid = widgets.FloatLogSlider(value=1e12,base=10,min=2,max=15,step=1,description='K')


In [5]:
text_0 = widgets.HTML(value="<h3>Founder strain replication time</h3>")
text_1 = widgets.HTML(value="<h3>Duplication strain replication time</h3>")
text_2= widgets.HTML(value="<h3>SNP strain replication time</h3>")
text_3= widgets.HTML(value="<h3>Mutation rate F to D</h3>")
text_4= widgets.HTML(value="<h3>Mutation rate D to F</h3>")
text_5= widgets.HTML(value="<h3>Mutation rate F to S</h3>")
text_6= widgets.HTML(value="<h3>Mutation rate S to F</h3>")
text_7= widgets.HTML(value="<h3>Carrying capacity</h3>")

In [7]:
vbox_text = widgets.VBox([text_0,tF, text_1, tD, text_2, tS, text_3, muftd,text_4, mudtf, text_5, mufts, text_6, mustf, text_7, Kwid])

In [8]:
def solving_function(tt, params, func, y0):
    n = len(y0)-1
    
    K,rF,rD,rS,mFD,mFS,mDF,mSF = params

    sol = odeint(func, y0, tt, args=(K,rF,rD,rS,mFD,mFS,mDF,mSF,), hmax=0.001)

    
    return tt, sol

In [9]:
def grad(y,t,K,rF,rD,rS,mFD,mFS,mDF,mSF):
    
    '''Gives the derivative with respect to time of the different subpopulations: R, Du, S'''
    
    dFdt = (1-(y[0]+y[1]+y[2])/K)*(rF*(1-mFD-mFS)*y[0]+rD*mDF*y[1]+rS*mSF*y[2])
    
    dDdt = (1-(y[0]+y[1]+y[2])/K)*(rD*(1-mDF)*y[1]+rF*mFD*y[0])
    
    dSdt = (1-(y[0]+y[1]+y[2])/K)*(rS*(1-mSF)*y[2]+rF*mFS*y[0])
    
    dydt = np.array([dFdt,dDdt,dSdt])
    
    return dydt

In [10]:
# button visualize 1

button_send = widgets.Button(
                description='Visualize',
                tooltip='Visualize',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        clear_output()
        
        rF = np.log(2)/tF.value
        rD = np.log(2)/tD.value
        rS = np.log(2)/tS.value
        
        mFD = muftd.value/np.log(2)
        mDF = mudtf.value/np.log(2)
        mFS = mufts.value/np.log(2)
        mSF = mustf.value/np.log(2)
        
        K = Kwid.value
        
        tcycle = 4
        bottleneck = 0.01
        ncycle = 10

        n0 = 1e5
        y0 = np.array([n0,0,0])

        t0temp = 0
        y0temp = y0
        tftemp = tcycle*ncycle
        tttemp = np.linspace(t0temp,tftemp,100)
        params = [K,rF,rD,rS,mFD,mFS,mDF,mSF]
        soltemp = solving_function(tttemp,params,grad,y0temp)
    
    
        ## Proportions
        solplot = np.transpose(soltemp[1])
        plt.plot(tttemp, solplot[0]/(solplot[0]+solplot[1]+solplot[2]),label='Founder strain')
        plt.plot(tttemp, solplot[1]/(solplot[0]+solplot[1]+solplot[2]),label='Duplication strain')
        plt.plot(tttemp, solplot[2]/(solplot[0]+solplot[1]+solplot[2]),label='SNP strain')
        
        plt.legend()
 
        plt.xlabel('time (days)')

        plt.ylabel('proportions')

        plt.tight_layout()
        
        A = plt.figure()
        
        ## Absolute numbers
        solplot = np.transpose(soltemp[1])
        plt.plot(tttemp, solplot[0],label='Founder strain')
        plt.plot(tttemp, solplot[1],label='Duplication strain')
        plt.plot(tttemp, solplot[2],label='SNP strain')
        
        plt.legend()
 
        plt.xlabel('time (days)')

        plt.ylabel('abundance')
        plt.semilogy()
        plt.tight_layout()
        
        B = plt.figure()  
        
        ## Simplex
        founder=solplot[0]/(solplot[0]+solplot[1]+solplot[2])
        dup=solplot[1]/(solplot[0]+solplot[1]+solplot[2])
        snp=solplot[2]/(solplot[0]+solplot[1]+solplot[2])
        
        scale = 1.0
        figure, tax = ternary.figure(scale=scale)
    
        tax.boundary(linewidth=2.0)
        tax.gridlines(color="blue", multiple=0.2)
        tax.set_title("Plotting of sample trajectory data", fontsize=20)
        points=[]
        for i in range(len(tttemp)):
            points.append((founder[i],dup[i],snp[i]))
        tax.plot(points, linewidth=2.0, label="Curve")
        fontsize = 12
        offset = 0.14
        tax.set_title("Various Lines\n", fontsize=fontsize)
        tax.right_corner_label("Founder", fontsize=fontsize)
        tax.top_corner_label("Dup", fontsize=fontsize)
        tax.left_corner_label("SNP", fontsize=fontsize)
    
        tax.ticks(axis='lbr', multiple=.2, linewidth=1, offset=0.025)
        tax.get_axes().axis('off')
        tax.clear_matplotlib_ticks()
        
        C = tax.show()
        
        return (A,B,C)
    

button_send.on_click(on_button_clicked)

vbox_result = widgets.HBox([button_send, output])

In [11]:
page = widgets.HBox([vbox_text,vbox_result])
display(page)

In [12]:
!pip freeze > requirements.txt